In [2]:
import pandas as pd
from joblib import load
import pickle
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier

# Load data

In [3]:
X_test = pd.read_csv("Data/X_test.csv")
y_test = pd.read_csv("Data/y_test.csv")

# Load models

In [13]:
class MLP_relu(nn.Module):
    def __init__(self, input_dim=10, num_hidden=1, hidden_dim=100, output_dim=4, dropout=0.5):
        """
        input_dim: Number of cells in the input layer
        num_hidden: Number of hidden layers
        hidden_dim: Number of cells in each hidden_layer
        output_dim: Number of cells in the output dimension 
        dropout: 
        """
        # Building the network from here
        super(MLP_relu, self).__init__()
        
        # Class attributes
        self.input_dim = input_dim
        self.num_hidden = num_hidden
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.dropout = nn.Dropout(dropout)
        
        # Hidden layers
        hidden = [nn.Linear(input_dim, hidden_dim) if i==0 else nn.Linear(hidden_dim, hidden_dim) for i in range(num_hidden)]
        self.linears = nn.ModuleList(hidden)
        
        # Output layer
        self.ol = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, data, **kwargs):
        # To float
        X = data.float()
        
        # Hidden layers
        for i, hl in enumerate(self.linears):
            X = self.linears[i](X)
            X = F.relu(X)
            X = self.dropout(X)
        
        # Output layer
        out = self.ol(X)
        out = F.softmax(out, dim = -1)
        
        return out

In [14]:
class MyNet(NeuralNetClassifier):
    def check_data(self, X, y):
        super().check_data(X, y)
        if self.module_.input_dim != X.shape[1]:
            self.set_params(module__input_dim=X.shape[1])
            self.initialize()

In [15]:
svm = load('Models/svm.joblib')
with open('Models/mlp.pkl', 'rb') as f:
    mlp = pickle.load(f)

## Making predictions

In [16]:
y_pred_svm = svm.predict(X_test.values.ravel())
y_pred_svm

array([1, 1, 3, ..., 0, 0, 0])

In [19]:
y_pred_mlp = mlp.predict(X_test.values.ravel())
y_pred_mlp

array([1, 1, 3, ..., 2, 3, 0])

## Weighted F1 score

In [9]:
from sklearn.metrics import f1_score

In [10]:
f1_svm = f1_score(y_test.values.ravel(), y_pred_svm, average='weighted')
f1_mlp = f1_score(y_test.values.ravel(), y_pred_mlp, average='weighted')

print("SVM F1-score:", f1_svm)
print("MLP F1-score:", f1_mlp)

SVM F1-score: 0.6646706960341937
MLP F1-score: 0.5807301589265353
